# J.P. Morgan Quantitative Research Experience
This online course gave me an insightful experience into a day in the life of someone carrying out a quant role at J.P. Morgan. The first task focused on predicting the price of oil, and then task 2 built on this, requiring us to derive the value of an oil contract using our future price predictions. Task 3 asked us to build a Probability of Default model, which took various customer factors such as number of credit lines and credit score and then output a probability of the customer not being able to pay back the loan. I experimented with regression but went for a Decision Tree and further a Random Forest, which takes the average of many Decision Trees. This gave less extreme probability estimates while still making very accurate predictions. Here I will go into task 4 in detail, which required us to quantise the FICO scores, grouping them into bands or ranges—to make them a stronger statistic when we use them in a Probability of Default function, for example.
# Task 4: Bucket FICO Scores
The input for our problem was a list of FICO scores and the number of buckets these needed to be filtered into. We needed to return optimised boundaries for these buckets such that scores in each respective bucket were as close to each other as possible. One method we were given to achieve this is by using the Mean Squared Error. We needed to find a bucket arrangement such that the total difference from each FICO score and it's respective Bucket midpoint is minimised. The MSE is a sufficient statistic for this.
$$
\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$
## Method

- **Step 1:** Sort the FICO scores and count how many times they appeared.
- **Step 2:** Compute the sum of squared errors (SSE) for every possible interval of unique scores.
- **Step 3:** We set up two matrices the first called dp which we fill with infinity (np.inf) because we are searching for a minimum SSE. If we find better we replace with our improved value. The second matrix bucket_index will store the index of where our last bucket needs to start for our most optimal arrangement.
- **Step 4:** We loop over each number of scores and number of buckets pair (i,k), each time we find the optimal SSE by splitting j scores into k-1 buckets, and the remaining scores into the kth bucket. We will always already have the optimal value of splitting j scores into k-1 buckets for all j less than i and so we just take the j with lowest total SSE
$$
\text{cost} = dp[j][k-1] + sse[j][i-1]
$$
- **Step 5:** Our bucket_index matrix now tells us where the last bucket should be to minimise SSE for each number of score/bucket combination. So from here we backtrack: find the index of where the last bucket should be for k buckets. We take the score at this index to find one of our boundaries and then go back to our table and find the last bucket_index for all scores below this boundary and k-1 buckets. This repeats until k=0. 

In [4]:
import pandas as pd
import numpy as np

loan_data = pd.read_csv('JP_Morgan/Task 3 and 4_Loan_Data.csv')
scores = np.sort(loan_data["fico_score"])

def quantize_fico(K):
    values, counts = np.unique(scores, return_counts=True)
    n = len(values)
    
    # We compute SSE for every interval of scores
    sse = np.zeros((n, n))
    for start in range(n):
        score_sum = 0
        total_count = 0
        for end in range(start, n):
            score_sum += values[end] * counts[end]
            total_count += counts[end]
            mean = score_sum / total_count
            sse[start][end] = np.sum(counts[start:end+1] * (values[start:end+1] - mean)**2)
    
    # DP table: dp[i][k] will be the minimum SSE for first i scores into k buckets
    dp = np.full((n+1, K+1), np.inf)
    #SSE for 0 scores and zero buckets is zero
    dp[0][0] = 0
    #bucket_index will give the index in scores where the last bucket starts for each i k combination
    bucket_index = np.zeros((n+1, K+1), dtype=int)
    
    #We find the optimal bucket arrangement for with the first scores and then use dynamic programming whilst adding more scores to find the optimal arrangement
    for i in range(1, n+1):
        for k in range(1, min(i, K)+1):
            for j in range(k-1, i):
                #cost is minimum SSE achievable with j scores and k-1 buckets + the SSE from if we put the bucket at index j
                cost = dp[j][k-1] + sse[j][i-1]
                #If this is the best SSE found so far we store the cost to beat and the index at where the last bucket should now be placed
                if cost < dp[i][k]:
                    dp[i][k] = cost
                    bucket_index[i][k] = j

    # Backtrack through the table to find bucket boundaries
    boundaries = []
    i, k = n, K
    while k > 0:
        j = bucket_index[i][k]
        boundaries.append((values[j], values[i-1]))
        i = j
        k = k-1
    boundaries.reverse()
    
    return boundaries

## Results
The test case below where we use 5 buckets gives us optimal boundaries to use for each bucket. This now gives us a rating system where if we had someone's credit score we can give it a specific number rating.

In [5]:
#Test case
K = 5  
boundaries = quantize_fico(K)
boundaries = [(int(low), int(high)) for low, high in boundaries]
print(boundaries)

[(408, 552), (553, 607), (608, 654), (655, 706), (707, 850)]


## Comparison
We'll quickly draft a simpler method where for 5 buckets we take 5 equidistant quantiles to divide the scores up and give us our boundaries. When we calculate the total MSE when using each method we are assured our method using dynamic programming is better, with an MSE about 53% smaller.

In [6]:
def total_mse(bucket_boundaries):
    total = 0
    for score in scores:
        for (low, high) in bucket_boundaries:
            if low <= score <= high:
                bucket_mid = (low + high) / 2
                total += (score - bucket_mid) ** 2
                break
    mse = total / len(scores)
    return mse
    
n = len(scores)
quantile_indices = [0, n//5, 2*n//5, 3*n//5, 4*n//5, n]
quantile_boundaries = []
bucket_mean = []
for i in range(5):
    bucket_scores = scores[quantile_indices[i]:quantile_indices[i+1]]
    min_score = bucket_scores[0]
    max_score = bucket_scores[-1]
    quantile_boundaries.append((min_score, max_score))
quantile_boundaries = [(int(low), int(high)) for low, high in quantile_boundaries]
print(total_mse(quantile_boundaries))
print(total_mse(boundaries))

1465.172675
689.416575
